In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from generate_mesh import *
from assembly import *
from post_processing import *
import pandas as pd
from plotting import *
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "serif"

## Constants

In [ ]:
# Wing structure dimensions
w = 5.46
h = 1.378
# Number of portions in the x
p = 25
# Number of portions in the y
m = 25
element_type = 'quadrilateral'

Cl = 0.5 
Cd = 0.01
rho = 0.770816
A = w*h
E = 69e9
nu = 0.33
V = np.linspace(51.4096, 76.8909, 100)

## Generating the mesh

In [ ]:
NL, EL = generate_mesh(w, h, p, m, element_type)

NoN = np.size(NL, 0)
NoE = np.size(EL, 0)

print(f'Number of nodes = {NoN}')
print(f'Number of elements = {NoE}')

plt.figure(1)

for i in range(0, NoN):
    plt.scatter(NL[i, 0], NL[i, 1], color = 'black')
#     plt.annotate(f'{i+1}', xy = [NL[i,0], NL[i,1]], fontsize=12)

if element_type == 'quadrilateral':
    x0, y0 = NL[EL[:, 0]-1, 0], NL[EL[:, 0]-1,1]
    x1, y1 = NL[EL[:, 1]-1, 0], NL[EL[:, 1]-1,1]
    x2, y2 = NL[EL[:, 2]-1, 0], NL[EL[:, 2]-1,1]
    x3, y3 = NL[EL[:, 3]-1, 0], NL[EL[:, 3]-1,1]

    plt.plot(np.array([x0, x1]), np.array([y0, y1]), 'k', linewidth=3)
    plt.plot(np.array([x1, x2]), np.array([y1, y2]), 'k', linewidth=3)
    plt.plot(np.array([x2, x3]), np.array([y2, y3]), 'k', linewidth=3)
    plt.plot(np.array([x3, x0]), np.array([y3, y0]), 'k', linewidth=3)

if element_type == 'triangular':
    x0, y0 = NL[EL[:, 0]-1, 0], NL[EL[:, 0]-1,1]
    x1, y1 = NL[EL[:, 1]-1, 0], NL[EL[:, 1]-1,1]
    x2, y2 = NL[EL[:, 2]-1, 0], NL[EL[:, 2]-1,1]
    plt.plot(np.array([x0, x1]), np.array([y0, y1]), 'k', linewidth=3)
    plt.plot(np.array([x1, x2]), np.array([y1, y2]), 'k', linewidth=3)
    plt.plot(np.array([x2, x0]), np.array([y2, y0]), 'k', linewidth=3)

plt.xlabel('x (m)', fontsize=12)
plt.ylabel('y (m)', fontsize=12)
plt.savefig('WingGeometry.png', dpi='figure')

## Assembling the global stiffness matrix and boundary conditions - Aluminum

In [ ]:
K = assemble_stiffness(NL, EL, E, nu)
nodes_BC = assign_BC(EL, p, m)
K_reduced = reduce_K(K, nodes_BC)
R = np.zeros([NoN*2, 1])

## Different loading conditions

In [ ]:
# For the distribution
u_net_dist_Al = np.zeros(len(V))
for i in range(len(V)):
    location, location_distribution = assign_forces(nodes_BC, p, m)
    D = -1/2 * rho * V[i]**2 * A * Cd
    R[location_distribution] = D/len(location_distribution)
    R_reduced = reduce_R(R, nodes_BC)
    u_reduced = np.linalg.inv(K_reduced)@R_reduced
    u = assemble_displacements(u_reduced, nodes_BC, NL, EL)
    u_dist = u
    u_net_dist_Al[i] = np.linalg.norm(u)

In [ ]:
# Single force
u_net_1_Al = np.zeros(len(V))
for i in range(len(V)):
    location, location_distribution = assign_forces(nodes_BC, p, m, 1)
    D = -1/2 * rho * V[i]**2 * A * Cd
    R[location] = D
    R_reduced = reduce_R(R, nodes_BC)
    u_reduced = np.linalg.inv(K_reduced)@R_reduced
    u = assemble_displacements(u_reduced, nodes_BC, NL, EL)
    u_1 = u
    u_net_1_Al[i] = np.linalg.norm(u)

In [ ]:
# Two forces
u_net_2_Al = np.zeros(len(V))
for i in range(len(V)):
    location, location_distribution = assign_forces(nodes_BC, p, m, 2)
    D = -1/2 * rho * V[i]**2 * A * Cd
    R[location] = D/len(location)
    R_reduced = reduce_R(R, nodes_BC)
    u_reduced = np.linalg.inv(K_reduced)@R_reduced
    u = assemble_displacements(u_reduced, nodes_BC, NL, EL)
    u_2 = u
    u_net_2_Al[i] = np.linalg.norm(u)

In [ ]:
# Three forces
u_net_3_Al = np.zeros(len(V))
for i in range(len(V)):
    location, location_distribution = assign_forces(nodes_BC, p, m, 3)
    D = -1/2 * rho * V[i]**2 * A * Cd
    R[location] = D/len(location)
    R_reduced = reduce_R(R, nodes_BC)
    u_reduced = np.linalg.inv(K_reduced)@R_reduced
    u = assemble_displacements(u_reduced, nodes_BC, NL, EL)
    u_3 = u
    u_net_3_Al[i] = np.linalg.norm(u)

## Post-Processing

In [ ]:
u_all = np.array([u_1, u_2, u_3, u_dist])
stress_yyNormalized_all = np.zeros([len(u), 4, NoE])
X_all = np.zeros([len(u), 4, NoE])
Y_all = np.zeros([len(u), 4, NoE])
for i in range(len(u_all)):
    (stress_xx, stress_xy, stress_yx, stress_yy,strain_xx,strain_xy,strain_yx,strain_yy, X, Y) = post_process(NL, EL, u_all[i], E, nu, 1e7)
    X_all[i] = X
    Y_all[i] = Y
    stress_yyNormalized = (stress_yy - stress_yy.min())/(stress_yy.max() - stress_yy.min())
    stress_yyNormalized_all[i] = stress_yyNormalized

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10,10), sharex=True, sharey=True)
for i in range(np.size(EL,0)):
    x = X_all[0][:,i]
    y = Y_all[0][:,i]
    c = stress_yyNormalized_all[0][:,i]
    cmap = truncate_colormap(plt.get_cmap('jet'), c.min(), c.max())
    t1 = ax[0,0].tripcolor(x, y, c, cmap = cmap, shading = 'gouraud')
    p = ax[0,0].plot(x,y,'k-', linewidth=0.5)
    ax[0,0].set_title('Case A')
    
    x = X_all[1][:,i]
    y = Y_all[1][:,i]
    c = stress_yyNormalized_all[1][:,i]
    cmap = truncate_colormap(plt.get_cmap('jet'), c.min(), c.max())
    t2 = ax[0,1].tripcolor(x, y, c, cmap = cmap, shading = 'gouraud')
    p = ax[0,1].plot(x,y,'k-', linewidth=0.5)
    ax[0,1].set_title('Case B')
    
    x = X_all[2][:,i]
    y = Y_all[2][:,i]
    c = stress_yyNormalized_all[2][:,i]
    cmap = truncate_colormap(plt.get_cmap('jet'), c.min(), c.max())
    t = ax[1,0].tripcolor(x, y, c, cmap = cmap, shading = 'gouraud')
    p = ax[1,0].plot(x,y,'k-', linewidth=0.5)
    ax[1,0].set_title('Case C')
    
    x = X_all[3][:,i]
    y = Y_all[3][:,i]
    c = stress_yyNormalized_all[3][:,i]
    cmap = truncate_colormap(plt.get_cmap('jet'), c.min(), c.max())
    t = ax[1,1].tripcolor(x, y, c, cmap = cmap, shading = 'gouraud')
    p = ax[1,1].plot(x,y,'k-', linewidth=0.5)
    ax[1,1].set_title('Case D')

for i in range(2):
    for j in range(2):
        ax[i, j].tick_params(axis='x', labelsize=11)
        ax[i, j].tick_params(axis='y', labelsize=11)

fig.add_subplot(111, frameon=False)
plt.tick_params(labelcolor='none', which='both', top=False, bottom=False, left=False, right=False)
        
plt.xlabel("x (m)", fontsize=12)
plt.ylabel("y (m)", fontsize=12)
    
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
fig.colorbar(t2, cax=cbar_ax)
fig.suptitle(r'$\sigma_{yy}$ - Aluminum', fontsize=25)
plt.savefig('AluminumStress.png', dpi='figure')

## Maximum displacement

In [ ]:
u_max = np.zeros(4)
u_max_nodes = np.zeros(4)
for i in range(len(u_max)):
    u_current = u_all[i]
    for j in range(int(len(u_current)/2)):
        current_max = 0
        current_max_node = 1
        ux_current = u_current[2*j]
        uy_current = u_current[2*j + 1]
        u_norm_current = np.sqrt(ux_current**2 + uy_current**2)
        if u_norm_current > current_max:
            current_max = u_norm_current
            current_max_node = j + 1
            print(current_max_node)
    u_max[i] = current_max
    u_max_nodes[i] = current_max_node

## Improved Design - Carbon Fiber

In [ ]:
# Wing structure dimensions
w = 5.46
h = 1.378
# Number of portions in the x
p = 25
# Number of portions in the y
m = 25
element_type = 'quadrilateral'

Cl = 0.5 
Cd = 0.01
rho = 0.770816
A = w*h
E = 135e9
nu = 0.3
V = np.linspace(51.4096, 76.8909, 100)

## Generating the mesh

In [ ]:
NL, EL = generate_mesh(w, h, p, m, element_type)

NoN = np.size(NL, 0)
NoE = np.size(EL, 0)

print(f'Number of nodes = {NoN}')
print(f'Number of elements = {NoE}')

plt.figure(1)

for i in range(0, NoN):
    plt.scatter(NL[i, 0], NL[i, 1], color = 'black')
#     plt.annotate(f'{i+1}', xy = [NL[i,0], NL[i,1]], fontsize=12)

if element_type == 'quadrilateral':
    x0, y0 = NL[EL[:, 0]-1, 0], NL[EL[:, 0]-1,1]
    x1, y1 = NL[EL[:, 1]-1, 0], NL[EL[:, 1]-1,1]
    x2, y2 = NL[EL[:, 2]-1, 0], NL[EL[:, 2]-1,1]
    x3, y3 = NL[EL[:, 3]-1, 0], NL[EL[:, 3]-1,1]

    plt.plot(np.array([x0, x1]), np.array([y0, y1]), 'k', linewidth=3)
    plt.plot(np.array([x1, x2]), np.array([y1, y2]), 'k', linewidth=3)
    plt.plot(np.array([x2, x3]), np.array([y2, y3]), 'k', linewidth=3)
    plt.plot(np.array([x3, x0]), np.array([y3, y0]), 'k', linewidth=3)

if element_type == 'triangular':
    x0, y0 = NL[EL[:, 0]-1, 0], NL[EL[:, 0]-1,1]
    x1, y1 = NL[EL[:, 1]-1, 0], NL[EL[:, 1]-1,1]
    x2, y2 = NL[EL[:, 2]-1, 0], NL[EL[:, 2]-1,1]
    plt.plot(np.array([x0, x1]), np.array([y0, y1]), 'k', linewidth=3)
    plt.plot(np.array([x1, x2]), np.array([y1, y2]), 'k', linewidth=3)
    plt.plot(np.array([x2, x0]), np.array([y2, y0]), 'k', linewidth=3)

plt.xlabel('x (m)', fontsize=12)
plt.ylabel('y (m)', fontsize=12)
plt.savefig('WingGeometry.png', dpi='figure')

## Assembling the global stiffness matrix and boundary conditions - Carbon Fiber

In [ ]:
K = assemble_stiffness(NL, EL, E, nu)
nodes_BC = assign_BC(EL, p, m)
K_reduced = reduce_K(K, nodes_BC)
R = np.zeros([NoN*2, 1])

## Different loading conditions

In [ ]:
# For the distribution
u_net_dist_carb = np.zeros(len(V))
for i in range(len(V)):
    location, location_distribution = assign_forces(nodes_BC, p, m)
    D = -1/2 * rho * V[i]**2 * A * Cd
    R[location_distribution] = D/len(location_distribution)
    R_reduced = reduce_R(R, nodes_BC)
    u_reduced = np.linalg.inv(K_reduced)@R_reduced
    u = assemble_displacements(u_reduced, nodes_BC, NL, EL)
    u_dist = u
    u_net_dist_carb[i] = np.linalg.norm(u)

In [ ]:
# Single force
u_net_1_carb = np.zeros(len(V))
for i in range(len(V)):
    location, location_distribution = assign_forces(nodes_BC, p, m, 1)
    D = -1/2 * rho * V[i]**2 * A * Cd
    R[location] = D
    R_reduced = reduce_R(R, nodes_BC)
    u_reduced = np.linalg.inv(K_reduced)@R_reduced
    u = assemble_displacements(u_reduced, nodes_BC, NL, EL)
    u_1 = u
    u_net_1_carb[i] = np.linalg.norm(u)

In [ ]:
# Two forces
u_net_2_carb = np.zeros(len(V))
for i in range(len(V)):
    location, location_distribution = assign_forces(nodes_BC, p, m, 2)
    D = -1/2 * rho * V[i]**2 * A * Cd
    R[location] = D/len(location)
    R_reduced = reduce_R(R, nodes_BC)
    u_reduced = np.linalg.inv(K_reduced)@R_reduced
    u = assemble_displacements(u_reduced, nodes_BC, NL, EL)
    u_2 = u
    u_net_2_carb[i] = np.linalg.norm(u)

In [ ]:
# Three forces
u_net_3_carb = np.zeros(len(V))
for i in range(len(V)):
    location, location_distribution = assign_forces(nodes_BC, p, m, 3)
    D = -1/2 * rho * V[i]**2 * A * Cd
    R[location] = D/len(location)
    R_reduced = reduce_R(R, nodes_BC)
    u_reduced = np.linalg.inv(K_reduced)@R_reduced
    u = assemble_displacements(u_reduced, nodes_BC, NL, EL)
    u_3 = u
    u_net_3_carb[i] = np.linalg.norm(u)

## Post-Processing

In [ ]:
u_all = np.array([u_1, u_2, u_3, u_dist])
stress_yyNormalized_all = np.zeros([len(u), 4, NoE])
X_all = np.zeros([len(u), 4, NoE])
Y_all = np.zeros([len(u), 4, NoE])
for i in range(len(u_all)):
    (stress_xx, stress_xy, stress_yx, stress_yy,strain_xx,strain_xy,strain_yx,strain_yy, X, Y) = post_process(NL, EL, u_all[i], E, nu, 1e7)
    X_all[i] = X
    Y_all[i] = Y
    stress_yyNormalized = (stress_yy - stress_yy.min())/(stress_yy.max() - stress_yy.min())
    stress_yyNormalized_all[i] = stress_yyNormalized

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10,10), sharex=True, sharey=True)
for i in range(np.size(EL,0)):
    x = X_all[0][:,i]
    y = Y_all[0][:,i]
    c = stress_yyNormalized_all[0][:,i]
    cmap = truncate_colormap(plt.get_cmap('jet'), c.min(), c.max())
    t1 = ax[0,0].tripcolor(x, y, c, cmap = cmap, shading = 'gouraud')
    p = ax[0,0].plot(x,y,'k-', linewidth=0.5)
    ax[0,0].set_title('Case A')
    
    x = X_all[1][:,i]
    y = Y_all[1][:,i]
    c = stress_yyNormalized_all[1][:,i]
    cmap = truncate_colormap(plt.get_cmap('jet'), c.min(), c.max())
    t2 = ax[0,1].tripcolor(x, y, c, cmap = cmap, shading = 'gouraud')
    p = ax[0,1].plot(x,y,'k-', linewidth=0.5)
    ax[0,1].set_title('Case B')
    
    x = X_all[2][:,i]
    y = Y_all[2][:,i]
    c = stress_yyNormalized_all[2][:,i]
    cmap = truncate_colormap(plt.get_cmap('jet'), c.min(), c.max())
    t = ax[1,0].tripcolor(x, y, c, cmap = cmap, shading = 'gouraud')
    p = ax[1,0].plot(x,y,'k-', linewidth=0.5)
    ax[1,0].set_title('Case C')
    
    x = X_all[3][:,i]
    y = Y_all[3][:,i]
    c = stress_yyNormalized_all[3][:,i]
    cmap = truncate_colormap(plt.get_cmap('jet'), c.min(), c.max())
    t = ax[1,1].tripcolor(x, y, c, cmap = cmap, shading = 'gouraud')
    p = ax[1,1].plot(x,y,'k-', linewidth=0.5)
    ax[1,1].set_title('Case D')

for i in range(2):
    for j in range(2):
        ax[i, j].tick_params(axis='x', labelsize=11)
        ax[i, j].tick_params(axis='y', labelsize=11)

fig.add_subplot(111, frameon=False)
plt.tick_params(labelcolor='none', which='both', top=False, bottom=False, left=False, right=False)
        
plt.xlabel("x (m)", fontsize=12)
plt.ylabel("y (m)", fontsize=12)
    
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
fig.colorbar(t2, cax=cbar_ax)
fig.suptitle(r'$\sigma_{yy}$ - Carbon Fiber', fontsize=25)
plt.savefig('CarbonFiberStress.png', dpi='figure')

## Plotting the displacement fields with each other and Improved Design

In [ ]:
u_net_Al = 1e3*np.array([u_net_1_Al, u_net_2_Al, u_net_3_Al, u_net_dist_Al])
u_net_carb = 1e3*np.array([u_net_1_carb, u_net_2_carb, u_net_3_carb, u_net_dist_carb])

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10,10), sharex=True)
fig.add_subplot(111, frameon=False)
plt.tick_params(labelcolor='none', which='both', top=False, bottom=False, left=False, right=False)
ax[0, 0].plot(V, u_net_Al[0], label=r'$\tilde{w}_{net} $ - Aluminum')
ax[0, 0].plot(V, u_net_carb[0], label=r'$\tilde{w}_{net} $ - Carbon Fiber')
ax[0, 0].set_title('Case A')

ax[0, 1].plot(V, u_net_Al[1], label=r'$\tilde{w}_{net} $ - Aluminum')
ax[0, 1].plot(V, u_net_carb[1], label=r'$\tilde{w}_{net} $ - Carbon Fiber')
ax[0, 1].set_title('Case B')

ax[1, 0].plot(V, u_net_Al[2], label=r'$\tilde{w}_{net} $ - Aluminum')
ax[1, 0].plot(V, u_net_carb[2], label=r'$\tilde{w}_{net} $ - Carbon Fiber')
ax[1, 0].set_title('Case C')

ax[1, 1].plot(V, u_net_Al[3], label=r'$\tilde{w}_{net} $ - Aluminum')
ax[1, 1].plot(V, u_net_carb[3], label=r'$\tilde{w}_{net} $ - Carbon Fiber')
ax[1, 1].set_title('Case D')

for i in range(2):
    for j in range(2):
        plt.ticklabel_format(axis='y', style='sci')
        ax[i, j].yaxis.major.formatter.set_powerlimits((0,0))
        ax[i, j].tick_params(axis='x', labelsize=11)
        ax[i, j].tick_params(axis='y', labelsize=11)
        ax[i, j].grid()
        ax[i, j].legend(fontsize=12)
plt.xlabel("Cruise speed (m/s)", fontsize=12)
plt.ylabel("Net displacement (mm)", fontsize=12)
fig.tight_layout()

## Convergence

In [ ]:
# Fix m, vary p
m = 3
p_dist = [i for i in range(1, p+20, 3)]
u_net_NoE = np.zeros(len(p_dist))

for i in range(len(p_dist)):
    p = p_dist[i]
    NL, EL = generate_mesh(w, h, p, m, element_type)
    NoN = np.size(NL, 0)
    NoE = np.size(EL, 0)
    K = assemble_stiffness(NL, EL, E, nu)
    nodes_BC = assign_BC(EL, p, m)
    K_reduced = reduce_K(K, nodes_BC)
    R = np.zeros([NoN*2, 1])
    location, location_distribution = assign_forces(nodes_BC, p, m)
    D = -1/2 * rho * V[-1]**2 * A * Cd
    R[location_distribution] = D/len(location_distribution)
    R_reduced = reduce_R(R, nodes_BC)
    u_reduced = np.linalg.inv(K_reduced)@R_reduced
    u = assemble_displacements(u_reduced, nodes_BC, NL, EL)
    u_net_NoE[i] = u.min()

In [ ]:
plt.plot(p_dist, u_net_NoE, '.')
plt.xlabel('Number of Elements across width', fontsize=12)
plt.ylabel('Maximum displacement (m)', fontsize=12)
plt.grid()
plt.savefig('Convergence.png')